In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn import preprocessing
from sklearn.linear_model.logistic import LogisticRegression
from sklearn.externals import joblib
from sklearn.preprocessing.data import OneHotEncoder
import warnings

warnings.filterwarnings("ignore")

import time

In [ ]:
def timestamp_datetime(value):
    format = '%Y-%m-%d %H:%M:%S'
    value = time.localtime(value)
    dt = time.strftime(format, value)
    return dt


def base_process(data):
    lbl = preprocessing.LabelEncoder()
    print('--------------------------item---------------------------------')
    data['len_item_category'] = data['item_category_list'].map(
        lambda x: len(str(x).split(';')))
    data['len_item_property'] = data['item_property_list'].map(
        lambda x: len(str(x).split(';')))
    for i in range(1, 2):
        data['item_category_list' + str(
            i
        )] = lbl.fit_transform(data['item_category_list'].map(
            lambda x: str(str(x).split(';')[i]) if len(str(x).split(';')) > i else ''
        ))  # item_category_list的第0列全部都一样
    for i in range(10):
        data['item_property_list' + str(
            i
        )] = lbl.fit_transform(data['item_property_list'].map(
            lambda x: str(str(x).split(';')[i]) if len(str(x).split(';')) > i else ''
        ))
    for col in ['item_id', 'item_brand_id', 'item_city_id']:
        data[col] = lbl.fit_transform(data[col])
    print('--------------------------user---------------------------------')
    for col in ['user_id']:
        data[col] = lbl.fit_transform(data[col])
    print('user 0,1 feature')
    data['gender0'] = data['user_gender_id'].apply(
        lambda x: 1 if x == -1 else 2)
    data['age0'] = data['user_age_level'].apply(
        lambda x: 1 if x == 1004 | x == 1005 | x == 1006 | x == 1007 else 2)
    data['occupation0'] = data['user_occupation_id'].apply(
        lambda x: 1 if x == -1 | x == 2003 else 2)
    data['star0'] = data['user_star_level'].apply(
        lambda x: 1 if x == -1 | x == 3000 | x == 3001 else 2)
    print(
        '--------------------context------------------------------------------'
    )
    data['realtime'] = data['context_timestamp'].apply(timestamp_datetime)
    data['realtime'] = pd.to_datetime(data['realtime'])
    data['day'] = data['realtime'].dt.day
    data['hour'] = data['realtime'].dt.hour
    data['len_predict_category_property'] = data[
        'predict_category_property'].map(lambda x: len(str(x).split(';')))

    for i in range(5):
        data['predict_category_property' + str(
            i
        )] = lbl.fit_transform(data['predict_category_property'].map(
            lambda x: str(str(x).split(';')[i]) if len(str(x).split(';')) > i else ''
        ))

    print('context 0,1 feature')
    data['context_page0'] = data['context_page_id'].apply(
        lambda x: 1 if x == 4001 | x == 4002 | x == 4003 | x == 4004 | x == 4007 else 2
    )
    print('-------------------------shop------------------------')
    for col in ['shop_id']:
        data[col] = lbl.fit_transform(data[col])
    data['shop_score_delivery0'] = data['shop_score_delivery'].apply(
        lambda x: 0 if x <= 0.98 and x >= 0.96 else 1)
    return data


def map_hour(x):
    if (x >= 0) & (x <= 7):
        return 1
    elif (x >= 13) & (x <= 18):
        return 2
    else:
        return 3


def deliver(x):
    #x=round(x,6)
    jiange = 0.1
    for i in range(1, 20):
        if (x >= 4.1 + jiange * (i - 1)) & (x <= 4.1 + jiange * i):
            return i + 1
    if x == -5:
        return 1


def deliver1(x):
    if (x >= 2) & (x <= 4):
        return 1
    elif (x >= 5) & (x <= 7):
        return 2
    else:
        return 3


def review(x):
    # x=round(x,6)
    jiange = 0.02
    for i in range(1, 30):
        if (x >= 0.714 + jiange * (i - 1)) & (x <= 0.714 + jiange * i):
            return i + 1
    if x == -1:
        return 1


def review1(x):
    # x=round(x,6)
    if (x >= 2) & (x <= 12):
        return 1
    elif (x >= 13) & (x <= 15):
        return 2
    else:
        return 3


def service(x):
    #x=round(x,6)
    jiange = 0.1
    for i in range(1, 20):
        if (x >= 3.93 + jiange * (i - 1)) & (x <= 3.93 + jiange * i):
            return i + 1
    if x == -1:
        return 1


def service1(x):
    if (x >= 2) & (x <= 7):
        return 1
    elif (x >= 8) & (x <= 9):
        return 2
    else:
        return 3


def describe(x):
    #x=round(x,6)
    jiange = 0.1
    for i in range(1, 30):
        if (x >= 3.93 + jiange * (i - 1)) & (x <= 3.93 + jiange * i):
            return i + 1
    if x == -1:
        return 1


def describe1(x):
    if (x >= 2) & (x <= 8):
        return 1
    elif (x >= 9) & (x <= 10):
        return 2
    else:
        return 3


def timeMap(data):
    data['hour_map'] = data['hour'].apply(map_hour)
    return data


def shop_box(data):
    data['shop_score_delivery'] = data['shop_score_delivery'] * 5
    data = data[data['shop_score_delivery'] != -5]
    data['deliver_map'] = data['shop_score_delivery'].apply(deliver)
    data['deliver_map'] = data['deliver_map'].apply(deliver1)
    # del data['shop_score_delivery']
    print(data.deliver_map.value_counts())

    data['shop_score_service'] = data['shop_score_service'] * 5
    data = data[data['shop_score_service'] != -5]
    data['service_map'] = data['shop_score_service'].apply(service)
    data['service_map'] = data['service_map'].apply(service1)
    # del data['shop_score_service']
    print(data.service_map.value_counts())  # 视为好评，中评，差评
    #
    data['shop_score_description'] = data['shop_score_description'] * 5
    data = data[data['shop_score_description'] != -5]
    data['de_map'] = data['shop_score_description'].apply(describe)
    data['de_map'] = data['de_map'].apply(describe1)
    # del data['shop_score_description']
    print(data.de_map.value_counts())

    data = data[data['shop_review_positive_rate'] != -1]
    data['review_map'] = data['shop_review_positive_rate'].apply(review)
    data['review_map'] = data['review_map'].apply(review1)
    print(data.review_map.value_counts())

    data['normal_shop'] = data.apply(
        lambda x: 1 if (x.deliver_map == 3) & (x.service_map == 3) & (x.de_map == 3) & (x.review_map == 3) else 0,
        axis=1)
    del data['de_map']
    del data['service_map']
    del data['deliver_map']
    del data['review_map']
    return data


def isPredict(df):

    DF2 = df[[
        'instance_id', 'predict_category_property', 'item_category_list',
        'item_property_list'
    ]]
    data = []
    for index, row in df.iterrows():
        categorySet = set(row['item_category_list'].split(';'))
        propertySet = set(row['item_property_list'].split(';'))
        predicts = row['predict_category_property'].split(';')

        preCate = []
        prePro = []
        for x in predicts:
            try:
                tmp = x.split(':')
                preCate.append(tmp[0])
                if tmp[1] != '-1':
                    prePro.append(tmp[1])
            except:
                print(index, x, row)

        isCategory = len(list(categorySet.intersection(set(preCate))))
        isProperty = len(list(propertySet.intersection(set(prePro))))

        data.append([row['instance_id'], isCategory, isProperty])

    data = pd.DataFrame(
        data, columns=['instance_id', 'isCategory', 'isProperty'])
    data = pd.merge(df, data, on=['instance_id'], how='left')

    return data


def slide_cnt(data):
    # item_cnt = data.groupby(by='item_id').count()['instance_id'].to_dict()
    # data['item_cnt'] = data['item_id'].apply(lambda x: item_cnt[x])
    # user_cnt = data.groupby(by='user_id').count()['instance_id'].to_dict()
    # data['user_cnt'] = data['user_id'].apply(lambda x: user_cnt[x])
    # shop_cnt = data.groupby(by='shop_id').count()['instance_id'].to_dict()
    # data['shop_cnt'] = data['shop_id'].apply(lambda x: shop_cnt[x])

    print('当前日期前一天的cnt')
    for d in range(19, 26):  # 18到24号
        df1 = data[data['day'] == d - 1]
        df2 = data[data['day'] == d]  # 19到25号
        user_cnt = df1.groupby(by='user_id').count()['instance_id'].to_dict()
        item_cnt = df1.groupby(by='item_id').count()['instance_id'].to_dict()
        shop_cnt = df1.groupby(by='shop_id').count()['instance_id'].to_dict()
        item_brand_cnt = df1.groupby(
            by='item_brand_id').count()['instance_id'].to_dict()
        item_city_cnt = df1.groupby(
            by='item_city_id').count()['instance_id'].to_dict()

        df2['user_cnt1'] = df2['user_id'].apply(lambda x: user_cnt.get(x, 0))
        df2['item_cnt1'] = df2['item_id'].apply(lambda x: item_cnt.get(x, 0))
        df2['shop_cnt1'] = df2['shop_id'].apply(lambda x: shop_cnt.get(x, 0))
        df2['item_brand_cnt1'] = df2['item_brand_id'].apply(
            lambda x: item_brand_cnt.get(x, 0))
        df2['item_city_cnt1'] = df2['item_city_id'].apply(
            lambda x: item_city_cnt.get(x, 0))

        df2 = df2[[
            'user_cnt1', 'item_cnt1', 'shop_cnt1', 'item_brand_cnt1',
            'item_city_cnt1', 'instance_id'
        ]]
        if d == 19:
            Df2 = df2
        else:
            Df2 = pd.concat([df2, Df2])
    data = pd.merge(data, Df2, on=['instance_id'], how='left')

    print('当前日期之前的cnt')
    for d in range(19, 26):
        # 19到25，25是test
        df1 = data[data['day'] < d]
        df2 = data[data['day'] == d]
        user_cnt = df1.groupby(by='user_id').count()['instance_id'].to_dict()
        item_cnt = df1.groupby(by='item_id').count()['instance_id'].to_dict()
        shop_cnt = df1.groupby(by='shop_id').count()['instance_id'].to_dict()
        shop_cnt = df1.groupby(by='shop_id').count()['instance_id'].to_dict()
        item_brand_cnt = df1.groupby(
            by='item_brand_id').count()['instance_id'].to_dict()
        item_city_cnt = df1.groupby(
            by='item_city_id').count()['instance_id'].to_dict()

        df2['user_cntx'] = df2['user_id'].apply(lambda x: user_cnt.get(x, 0))
        df2['item_cntx'] = df2['item_id'].apply(lambda x: item_cnt.get(x, 0))
        df2['shop_cntx'] = df2['shop_id'].apply(lambda x: shop_cnt.get(x, 0))
        df2['item_brand_cntx'] = df2['item_brand_id'].apply(
            lambda x: item_brand_cnt.get(x, 0))
        df2['item_city_cntx'] = df2['item_city_id'].apply(
            lambda x: item_city_cnt.get(x, 0))
        df2 = df2[[
            'user_cntx', 'item_cntx', 'shop_cntx', 'item_brand_cntx',
            'item_city_cntx', 'instance_id'
        ]]
        if d == 19:
            Df2 = df2
        else:
            Df2 = pd.concat([df2, Df2])
    data = pd.merge(data, Df2, on=['instance_id'], how='left')

    #print("前一个小时的统计量")

    return data


def join(data):
    for col in [
            'item_sales_level', 'item_price_level', 'item_collected_level',
            'user_gender_id', 'user_age_level', 'user_occupation_id',
            'user_star_level', 'shop_review_num_level', 'shop_star_level',
            'hour', 'item_category_list1'
    ]:
        data[col] = data[col].apply(lambda x: 0 if x == -1 else x)

    for col in [
            'item_sales_level', 'item_price_level', 'item_collected_level',
            'user_gender_id', 'user_age_level', 'user_occupation_id',
            'user_star_level', 'shop_review_num_level', 'shop_star_level',
            'hour', 'item_category_list1'
    ]:
        data[col] = data[col].astype(str)

    print('item两两组合')
    data = data.assign(A=data.item_sales_level + data.item_price_level)
    data = data.assign(B=data.item_sales_level + data.item_collected_level)
    data = data.assign(C=data.item_price_level + data.item_collected_level)

    print('user两两组合')
    data = data.assign(D=data.user_gender_id + data.item_price_level)
    data = data.assign(E=data.user_gender_id + data.user_occupation_id)
    data = data.assign(F=data.user_gender_id + data.user_star_level)
    data = data.assign(G=data.user_gender_id + data.hour)
    data = data.assign(H=data.user_occupation_id + data.hour)

    print('shop两两组合')
    data = data.assign(I=data.shop_review_num_level + data.shop_star_level)

    print('shop-item两两组合')
    data = data.assign(J=data.shop_review_num_level + data.item_sales_level)
    data = data.assign(K=data.shop_review_num_level + data.item_price_level)
    data = data.assign(
        L=data.shop_review_num_level + data.item_collected_level)
    data = data.assign(M=data.shop_star_level + data.item_sales_level)
    data = data.assign(N=data.shop_star_level + data.item_price_level)
    data = data.assign(O=data.shop_star_level + data.item_collected_level)

    print('类别时间两两组合')
    data = data.assign(P=data.item_category_list1 + data.hour)

    for col in [
            'item_sales_level', 'item_price_level', 'item_collected_level',
            'user_gender_id', 'user_age_level', 'user_occupation_id',
            'user_star_level', 'shop_review_num_level', 'shop_star_level',
            'hour', 'item_category_list1', 'A', 'B', 'C', 'D', 'E', 'F', 'G',
            'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P'
    ]:
        data[col] = data[col].astype(int)

    return data


def item(data):
    print('一个item有多少brand,price salse collected level……')

    itemcnt = data.groupby(
        ['item_id'], as_index=False)['instance_id'].agg({
            'item_cnt': 'count'
        })
    data = pd.merge(data, itemcnt, on=['item_id'], how='left')

    for col in [
            'item_brand_id', 'item_city_id', 'item_price_level',
            'item_sales_level', 'item_collected_level', 'item_pv_level'
    ]:
        itemcnt = data.groupby(
            [col, 'item_id'], as_index=False)['instance_id'].agg({
                str(col) + '_item_cnt':
                'count'
            })
        data = pd.merge(data, itemcnt, on=[col, 'item_id'], how='left')
        data[str(col) + '_item_prob'] = data[str(col)
                                             + '_item_cnt'] / data['item_cnt']
    del data['item_cnt']

    print('一个brand有多少price salse collected level……')

    itemcnt = data.groupby(
        ['item_brand_id'], as_index=False)['instance_id'].agg({
            'item_brand_cnt':
            'count'
        })
    data = pd.merge(data, itemcnt, on=['item_brand_id'], how='left')

    for col in [
            'item_city_id', 'item_price_level', 'item_sales_level',
            'item_collected_level', 'item_pv_level'
    ]:
        itemcnt = data.groupby(
            [col, 'item_brand_id'], as_index=False)['instance_id'].agg({
                str(col) + '_brand_cnt':
                'count'
            })
        data = pd.merge(data, itemcnt, on=[col, 'item_brand_id'], how='left')
        data[str(col) + '_brand_prob'] = data[str(col) + '_brand_cnt'] / data[
            'item_brand_cnt']
    del data['item_brand_cnt']

    print(
        '一个city有多少item_price_level，item_sales_level，item_collected_level，item_pv_level'
    )

    itemcnt = data.groupby(
        ['item_city_id'], as_index=False)['instance_id'].agg({
            'item_city_cnt':
            'count'
        })
    data = pd.merge(data, itemcnt, on=['item_city_id'], how='left')
    for col in [
            'item_price_level', 'item_sales_level', 'item_collected_level',
            'item_pv_level'
    ]:
        itemcnt = data.groupby(
            [col, 'item_city_id'], as_index=False)['instance_id'].agg({
                str(col) + '_city_cnt':
                'count'
            })
        data = pd.merge(data, itemcnt, on=[col, 'item_city_id'], how='left')
        data[str(col) + '_city_prob'] = data[str(col) + '_city_cnt'] / data[
            'item_city_cnt']
    del data['item_city_cnt']

    print('一个price有多少item_sales_level，item_collected_level，item_pv_level')

    itemcnt = data.groupby(
        ['item_price_level'], as_index=False)['instance_id'].agg({
            'item_price_cnt':
            'count'
        })
    data = pd.merge(data, itemcnt, on=['item_price_level'], how='left')
    for col in ['item_sales_level', 'item_collected_level', 'item_pv_level']:
        itemcnt = data.groupby(
            [col, 'item_city_id'], as_index=False)['instance_id'].agg({
                str(col) + '_price_cnt':
                'count'
            })
        data = pd.merge(data, itemcnt, on=[col, 'item_city_id'], how='left')
        data[str(col) + '_price_prob'] = data[str(col) + '_price_cnt'] / data[
            'item_price_cnt']
    del data['item_price_cnt']

    print('一个item_sales_level有多少item_collected_level，item_pv_level')

    itemcnt = data.groupby(
        ['item_sales_level'], as_index=False)['instance_id'].agg({
            'item_salse_cnt':
            'count'
        })
    data = pd.merge(data, itemcnt, on=['item_sales_level'], how='left')
    for col in ['item_collected_level', 'item_pv_level']:
        itemcnt = data.groupby(
            [col, 'item_sales_level'], as_index=False)['instance_id'].agg({
                str(col) + '_salse_cnt':
                'count'
            })
        data = pd.merge(
            data, itemcnt, on=[col, 'item_sales_level'], how='left')
        data[str(col) + '_salse_prob'] = data[str(col) + '_salse_cnt'] / data[
            'item_salse_cnt']
    del data['item_salse_cnt']

    print('一个item_collected_level有多少item_pv_level')

    itemcnt = data.groupby(
        ['item_collected_level'], as_index=False)['instance_id'].agg({
            'item_coll_cnt':
            'count'
        })
    data = pd.merge(data, itemcnt, on=['item_collected_level'], how='left')
    for col in ['item_pv_level']:
        itemcnt = data.groupby(
            [col, 'item_collected_level'], as_index=False)['instance_id'].agg({
                str(col) + '_coll_cnt':
                'count'
            })
        data = pd.merge(
            data, itemcnt, on=[col, 'item_collected_level'], how='left')
        data[str(col) + '_coll_prob'] = data[str(col) + '_coll_cnt'] / data[
            'item_coll_cnt']
    del data['item_coll_cnt']

    return data


def user(data):
    print('用户有多少性别')
    itemcnt = data.groupby(
        ['user_id'], as_index=False)['instance_id'].agg({
            'user_cnt': 'count'
        })
    data = pd.merge(data, itemcnt, on=['user_id'], how='left')

    for col in [
            'user_gender_id', 'user_age_level', 'user_occupation_id',
            'user_star_level'
    ]:
        itemcnt = data.groupby(
            [col, 'user_id'], as_index=False)['instance_id'].agg({
                str(col) + '_user_cnt':
                'count'
            })
        data = pd.merge(data, itemcnt, on=[col, 'user_id'], how='left')
        data[str(col) + '_user_prob'] = data[str(col)
                                             + '_user_cnt'] / data['user_cnt']
    del data['user_cnt']

    print('性别的年龄段，职业有多少')
    itemcnt = data.groupby(
        ['user_gender_id'], as_index=False)['instance_id'].agg({
            'user_gender_cnt':
            'count'
        })
    data = pd.merge(data, itemcnt, on=['user_gender_id'], how='left')

    for col in ['user_age_level', 'user_occupation_id', 'user_star_level']:
        itemcnt = data.groupby(
            [col, 'user_gender_id'], as_index=False)['instance_id'].agg({
                str(col) + '_user_gender_cnt':
                'count'
            })
        data = pd.merge(data, itemcnt, on=[col, 'user_gender_id'], how='left')
        data[str(col) + '_user_gender_prob'] = data[
            str(col) + '_user_gender_cnt'] / data['user_gender_cnt']
    del data['user_gender_cnt']

    print('user_age_level对应的user_occupation_id，user_star_level')
    itemcnt = data.groupby(
        ['user_age_level'], as_index=False)['instance_id'].agg({
            'user_age_cnt':
            'count'
        })
    data = pd.merge(data, itemcnt, on=['user_age_level'], how='left')

    for col in ['user_occupation_id', 'user_star_level']:
        itemcnt = data.groupby(
            [col, 'user_age_level'], as_index=False)['instance_id'].agg({
                str(col) + '_user_age_cnt':
                'count'
            })
        data = pd.merge(data, itemcnt, on=[col, 'user_age_level'], how='left')
        data[str(col) + '_user_age_prob'] = data[
            str(col) + '_user_age_cnt'] / data['user_age_cnt']
    del data['user_age_cnt']

    print('user_occupation_id对应的user_star_level')
    itemcnt = data.groupby(
        ['user_occupation_id'], as_index=False)['instance_id'].agg({
            'user_occ_cnt':
            'count'
        })
    data = pd.merge(data, itemcnt, on=['user_occupation_id'], how='left')
    for col in ['user_star_level']:
        itemcnt = data.groupby(
            [col, 'user_occupation_id'], as_index=False)['instance_id'].agg({
                str(col) + '_user_occ_cnt':
                'count'
            })
        data = pd.merge(
            data, itemcnt, on=[col, 'user_occupation_id'], how='left')
        data[str(col) + '_user_occ_prob'] = data[
            str(col) + '_user_occ_cnt'] / data['user_occ_cnt']
    del data['user_occ_cnt']

    return data


def user_item(data):
    itemcnt = data.groupby(
        ['user_id'], as_index=False)['instance_id'].agg({
            'user_cnt': 'count'
        })
    data = pd.merge(data, itemcnt, on=['user_id'], how='left')
    print('一个user有多少item_id,item_brand_id……')
    for col in [
            'item_id', 'item_brand_id', 'item_city_id', 'item_price_level',
            'item_sales_level', 'item_collected_level', 'item_pv_level'
    ]:
        item_shop_cnt = data.groupby(
            [col, 'user_id'], as_index=False)['instance_id'].agg({
                str(col) + '_user_cnt':
                'count'
            })
        data = pd.merge(data, item_shop_cnt, on=[col, 'user_id'], how='left')
        data[str(col) + '_user_prob'] = data[str(col)
                                             + '_user_cnt'] / data['user_cnt']

    print('一个user_gender有多少item_id,item_brand_id……')
    itemcnt = data.groupby(
        ['user_gender_id'], as_index=False)['instance_id'].agg({
            'user_gender_cnt':
            'count'
        })
    data = pd.merge(data, itemcnt, on=['user_gender_id'], how='left')
    for col in [
            'item_id', 'item_brand_id', 'item_city_id', 'item_price_level',
            'item_sales_level', 'item_collected_level', 'item_pv_level'
    ]:
        item_shop_cnt = data.groupby(
            [col, 'user_gender_id'], as_index=False)['instance_id'].agg({
                str(col) + '_user_gender_cnt':
                'count'
            })
        data = pd.merge(
            data, item_shop_cnt, on=[col, 'user_gender_id'], how='left')
        data[str(col) + '_user_gender_prob'] = data[
            str(col) + '_user_gender_cnt'] / data['user_gender_cnt']

    print('一个user_age_level有多少item_id,item_brand_id……')
    itemcnt = data.groupby(
        ['user_age_level'], as_index=False)['instance_id'].agg({
            'user_age_cnt':
            'count'
        })
    data = pd.merge(data, itemcnt, on=['user_age_level'], how='left')
    for col in [
            'item_id', 'item_brand_id', 'item_city_id', 'item_price_level',
            'item_sales_level', 'item_collected_level', 'item_pv_level'
    ]:
        item_shop_cnt = data.groupby(
            [col, 'user_age_level'], as_index=False)['instance_id'].agg({
                str(col) + '_user_age_cnt':
                'count'
            })
        data = pd.merge(
            data, item_shop_cnt, on=[col, 'user_age_level'], how='left')
        data[str(col) + '_user_age_prob'] = data[
            str(col) + '_user_age_cnt'] / data['user_age_cnt']

    print('一个user_occupation_id有多少item_id,item_brand_id…')
    itemcnt = data.groupby(
        ['user_occupation_id'], as_index=False)['instance_id'].agg({
            'user_occ_cnt':
            'count'
        })
    data = pd.merge(data, itemcnt, on=['user_occupation_id'], how='left')
    for col in [
            'item_id', 'item_brand_id', 'item_city_id', 'item_price_level',
            'item_sales_level', 'item_collected_level', 'item_pv_level'
    ]:
        item_shop_cnt = data.groupby(
            [col, 'user_occupation_id'], as_index=False)['instance_id'].agg({
                str(col) + '_user_occ_cnt':
                'count'
            })
        data = pd.merge(
            data, item_shop_cnt, on=[col, 'user_occupation_id'], how='left')
        data[str(col) + '_user_occ_prob'] = data[
            str(col) + '_user_occ_cnt'] / data['user_occ_cnt']

    return data


def user_shop(data):
    print('一个user有多少shop_id,shop_review_num_level……')

    for col in ['shop_id', 'shop_review_num_level', 'shop_star_level']:
        item_shop_cnt = data.groupby(
            [col, 'user_id'], as_index=False)['instance_id'].agg({
                str(col) + '_user_cnt':
                'count'
            })
        data = pd.merge(data, item_shop_cnt, on=[col, 'user_id'], how='left')
        data[str(col) + '_user_prob'] = data[str(col)
                                             + '_user_cnt'] / data['user_cnt']
    del data['user_cnt']

    print('一个user_gender有多少shop_id,shop_review_num_level……')
    for col in ['shop_id', 'shop_review_num_level', 'shop_star_level']:
        item_shop_cnt = data.groupby(
            [col, 'user_gender_id'], as_index=False)['instance_id'].agg({
                str(col) + '_user_gender_cnt':
                'count'
            })
        data = pd.merge(
            data, item_shop_cnt, on=[col, 'user_gender_id'], how='left')
        data[str(col) + '_user_gender_prob'] = data[
            str(col) + '_user_gender_cnt'] / data['user_gender_cnt']
    del data['user_gender_cnt']

    print('一个user_age_level有多少shop_id,shop_review_num_level……')
    for col in ['shop_id', 'shop_review_num_level', 'shop_star_level']:
        item_shop_cnt = data.groupby(
            [col, 'user_age_level'], as_index=False)['instance_id'].agg({
                str(col) + '_user_age_cnt':
                'count'
            })
        data = pd.merge(
            data, item_shop_cnt, on=[col, 'user_age_level'], how='left')
        data[str(col) + '_user_age_prob'] = data[
            str(col) + '_user_age_cnt'] / data['user_age_cnt']
    del data['user_age_cnt']

    print('一个user_occupation_id有多少shop_id,shop_review_num_level……')
    for col in ['shop_id', 'shop_review_num_level', 'shop_star_level']:
        item_shop_cnt = data.groupby(
            [col, 'user_occupation_id'], as_index=False)['instance_id'].agg({
                str(col) + '_user_occ_cnt':
                'count'
            })
        data = pd.merge(
            data, item_shop_cnt, on=[col, 'user_occupation_id'], how='left')
        data[str(col) + '_user_occ_prob'] = data[
            str(col) + '_user_occ_cnt'] / data['user_occ_cnt']
    del data['user_occ_cnt']

    return data


def shop_item(data):
    print('一个shop有多少item_id,item_brand_id,item_city_id,item_price_level……')
    itemcnt = data.groupby(
        ['shop_id'], as_index=False)['instance_id'].agg({
            'shop_cnt': 'count'
        })
    data = pd.merge(data, itemcnt, on=['shop_id'], how='left')
    for col in [
            'item_id', 'item_brand_id', 'item_city_id', 'item_price_level',
            'item_sales_level', 'item_collected_level', 'item_pv_level'
    ]:
        item_shop_cnt = data.groupby(
            [col, 'shop_id'], as_index=False)['instance_id'].agg({
                str(col) + '_shop_cnt':
                'count'
            })
        data = pd.merge(data, item_shop_cnt, on=[col, 'shop_id'], how='left')
        data[str(col) + '_shop_prob'] = data[str(col)
                                             + '_shop_cnt'] / data['shop_cnt']
    del data['shop_cnt']

    print(
        '一个shop_review_num_level有多少item_id,item_brand_id,item_city_id,item_price_level……'
    )
    itemcnt = data.groupby(
        ['shop_review_num_level'], as_index=False)['instance_id'].agg({
            'shop_rev_cnt':
            'count'
        })
    data = pd.merge(data, itemcnt, on=['shop_review_num_level'], how='left')
    for col in [
            'item_id', 'item_brand_id', 'item_city_id', 'item_price_level',
            'item_sales_level', 'item_collected_level', 'item_pv_level'
    ]:
        item_shop_cnt = data.groupby(
            [col, 'shop_review_num_level'],
            as_index=False)['instance_id'].agg({
                str(col) + '_shop_rev_cnt':
                'count'
            })
        data = pd.merge(
            data, item_shop_cnt, on=[col, 'shop_review_num_level'], how='left')
        data[str(col) + '_shop_rev_prob'] = data[
            str(col) + '_shop_rev_cnt'] / data['shop_rev_cnt']
    del data['shop_rev_cnt']

    # print('一个shop_star_level有多少item_id,item_brand_id,item_city_id,item_price_level……')
    # itemcnt = data.groupby(['shop_star_level'], as_index=False)['instance_id'].agg({'shop_star_cnt': 'count'})
    # data = pd.merge(data, itemcnt, on=['shop_star_level'], how='left')
    # for col in ['item_id',
    #             'item_brand_id', 'item_city_id', 'item_price_level',
    #             'item_sales_level', 'item_collected_level', 'item_pv_level']:
    #     item_shop_cnt = data.groupby([col, 'shop_star_level'], as_index=False)['instance_id'].agg({str(col) + '_shop_star_cnt': 'count'})
    #     data = pd.merge(data, item_shop_cnt, on=[col, 'shop_star_level'], how='left')
    #     data[str(col) + '_shop_star_prob'] = data[str(col) + '_shop_star_cnt'] / data['shop_star_cnt']
    # del data['shop_star_cnt']
    return data


def lgbCV_online(train, test):
    col = [
        c for c in train if c not in [
            'is_trade', 'item_category_list', 'item_property_list',
            'predict_category_property', 'instance_id', 'context_id',
            'realtime', 'context_timestamp'
        ]
    ]
    # cat = ['sale_price', 'gender_star', 'user_age_level', 'item_price_level', 'item_sales_level', 'sale_collect',
    #        'price_collect', 'item_brand_id', 'user_star_level', 'item_id', 'shop_id',
    #        'item_city_id', 'context_page_id', 'gender_age', 'shop_star_level', 'item_pv_level', 'user_occupation_id',
    #        'day', 'gender_occ', 'user_gender_id']
    X = train[col]
    y = train['is_trade'].values
    X_tes = test[col]
    y_tes = test['is_trade'].values
    print('Training LGBM model...')
    lgb0 = lgb.LGBMClassifier(
        objective='binary',
        # metric='binary_error',
        num_leaves=35,
        depth=8,
        learning_rate=0.005,
        seed=2018,
        colsample_bytree=0.8,
        # min_child_samples=8,
        subsample=0.9,
        n_estimators=20000)
    lgb_model = lgb0.fit(
        X, y, eval_set=[(X_tes, y_tes)], early_stopping_rounds=200)
    best_iter = lgb_model.best_iteration_

    #print(get_leaf_output(tree_id, leaf_id))
    '''
    predictors = [i for i in X.columns]
    feat_imp = pd.Series(lgb_model.feature_importance(), predictors).sort_values(ascending=False)
    print(feat_imp)
    print(feat_imp.shape)
    # pred= lgb_model.predict(test[col])
    '''
    pred = lgb_model.predict_proba(test[col])[:, 1]
    test['pred'] = pred
    test['index'] = range(len(test))
    # print(test[['is_trade','pred']])
    print('误差 ', log_loss(test['is_trade'], test['pred']))

    # +lr
    # 编码原有特征
    X_train_leaves = lgb0.apply(X)
    X_test_leaves = lgb0.apply(X_tes)

    # 对所有特征进行ont-hot编码
    (train_rows, cols) = X_train_leaves.shape

    gbdtenc = OneHotEncoder()
    X_trans = gbdtenc.fit_transform(
        np.concatenate((X_train_leaves, X_test_leaves), axis=0))

    # 定义LR模型
    lr = LogisticRegression()

    # lr对gbdt特征编码后的样本模型训练
    lr.fit(X_trans[:train_rows, :], y)

    # 预测及AUC评测
    y_pred_gbdtlr = lr.predict_proba(X_trans[train_rows:, :])[:, 1]
    #     gbdt_lr_test_log_loss = log_loss(y_tes, y_pred_gbdtlr)
    #     print('基于GBDT特征编码后的LR log_loss: %.5f' % gbdt_lr_test_log_loss)

    return best_iter, y_tes, y_pred_gbdtlr


def sub_online(train, test, best_iter, path):
    col = [
        c for c in train if c not in [
            'is_trade', 'item_category_list', 'item_property_list',
            'predict_category_property', 'instance_id', 'context_id',
            'realtime', 'context_timestamp'
        ]
    ]
    X = train[col]
    y = train['is_trade'].values
    print('Training LGBM model...')
    lgb0 = lgb.LGBMClassifier(
        objective='binary',
        # metric='binary_error',
        num_leaves=35,
        depth=8,
        learning_rate=0.05,
        seed=2018,
        colsample_bytree=0.8,
        # min_child_samples=8,
        subsample=0.9,
        n_estimators=best_iter)
    lgb_model = lgb0.fit(X, y)

    # pred= lgb_model.predict(test[col])
    pred = lgb_model.predict_proba(test[col])[:, 1]
    test['predicted_score'] = pred
    sub1 = test[['instance_id', 'predicted_score']]
    sub = pd.read_csv(path + "round1_ijcai_18_test_a_20180301.txt", sep="\s+")
    sub = pd.merge(sub, sub1, on=['instance_id'], how='left')
    sub = sub.fillna(0)
    return sub[['instance_id', 'predicted_score']]

In [ ]:
path = '../Data/'
train = pd.read_csv(path + 'round1_ijcai_18_train_20180301.txt', sep="\s+")
test = pd.read_csv(path + 'round1_ijcai_18_test_a_20180301.txt', sep="\s+")
data = pd.concat([train, test])
data = data.drop_duplicates(subset='instance_id')  # 把instance id去重
print('make feature')
data = base_process(data)
data = timeMap(data)
data = shop_box(data)
data = slide_cnt(data)
#data = isPredict(data)
data = join(data)
print('-------------------------全局统计特征------------------------')
data = item(data)
data = user(data)
data = user_item(data)
data = user_shop(data)
data = shop_item(data)

make feature
--------------------------item---------------------------------
--------------------------user---------------------------------
user 0,1 feature
--------------------context------------------------------------------
context 0,1 feature
-------------------------shop------------------------
3    493224
2      3146
1        25
Name: deliver_map, dtype: int64
3    490039
2      6243
1       113
Name: service_map, dtype: int64
3    394629
2     99788
1      1978
Name: de_map, dtype: int64
3    356965
2    138783
1       647
Name: review_map, dtype: int64
当前日期前一天的cnt
当前日期之前的cnt
item两两组合
user两两组合
shop两两组合
shop-item两两组合
类别时间两两组合
-------------------------全局统计特征------------------------
一个item有多少brand,price salse collected level……
一个brand有多少price salse collected level……
一个city有多少item_price_level，item_sales_level，item_collected_level，item_pv_level
一个price有多少item_sales_level，item_collected_level，item_pv_level
一个item_sales_level有多少item_collected_level，item_pv_level
一个item_collected_level

In [ ]:
print("-------------------------线下---------------------------")
# 0~7小时
train1 = data[(data['day'] >= 18) & (data['day'] <= 23) &
              (data['hour_map'] == 1)]
test1 = data[(data['day'] == 24) & (data['hour_map'] == 1)]
best_iter, y_tes1, y_pred_gbdtlr1 = lgbCV_online(train1, test1)
joblib.dump(best_iter, 'gdbt_lr_0to7.model')
print("0~7小时模型完成")
# 8~15小时
train2 = data[(data['day'] >= 18) & (data['day'] <= 23) &
              (data['hour_map'] == 2)]
test2 = data[(data['day'] == 24) & (data['hour_map'] == 2)]
best_iter, y_tes2, y_pred_gbdtlr2 = lgbCV_online(train2, test2)
joblib.dump(best_iter, 'gdbt_lr_8to15.model')
print("8~15小时模型完成")
# 15~23小时
train3 = data[(data['day'] >= 18) & (data['day'] <= 23) &
              (data['hour_map'] == 3)]
test3 = data[(data['day'] == 24) & (data['hour_map'] == 3)]
best_iter, y_tes3, y_pred_gbdtlr3 = lgbCV_online(train3, test3)
joblib.dump(best_iter, 'gdbt_lr_16to23.model')
print("16~23小时模型完成")

y_tes = pd.concat([y_tes1, y_tes2, y_tes3], ignore_index=True)
y_pred_gbdtlr = pd.concat(
    [y_pred_gbdtlr1, y_pred_gbdtlr2, y_pred_gbdtlr3], ignore_index=True)

gbdt_lr_test_log_loss = log_loss(y_tes, y_pred_gbdtlr)
print('基于GBDT特征编码后的LR log_loss: %.5f' % gbdt_lr_test_log_loss)

In [ ]:
print("-------------------线上----------------------")
train1 = data[data.is_trade.notnull() & (data['hour_map'] == 1)]
test1 = data[data.is_trade.isnull() & (data['hour_map'] == 1)]
gdbt_lr = joblib.load('gdbt_lr_0to7.model')
res1 = sub_online(train1, test1, gdbt_lr, path)

train2 = data[data.is_trade.notnull() & (data['hour_map'] == 2)]
test2 = data[data.is_trade.isnull() & (data['hour_map'] == 2)]
gdbt_lr = joblib.load('gdbt_lr_8to15.model')
res2 = sub_online(train2, test2, gdbt_lr, path)

train3 = data[data.is_trade.notnull() & (data['hour_map'] == 3)]
test3 = data[data.is_trade.isnull() & (data['hour_map'] == 3)]
gdbt_lr = joblib.load('gdbt_lr_16to23.model')
res3 = sub_online(train3, test3, gdbt_lr, path)

res = pd.concat([res1, res2, res3], ignore_index=True)
res.to_csv('../result_0406.txt',sep=" ",index=False)